In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from scipy.fftpack import rfft, fftfreq, fft
import scipy.sparse as sparse
import scipy.linalg as linalg
from scipy.sparse.linalg import LinearOperator
import numpy as np
import collections
import copy
from amfe.utils.utils import OrderedSet
from amfe.cyclic.cyclic import SelectionOperator
import amfe
from amfe.frequency_module.frequency import assemble_HBMOperator, create_Z_matrix
from amfe.operators.operators import build_SelectionOperator 
#from scipy import optimize
from amfe.optimize import optimize
#unpack case dict

msh_file = r'meshes\2D_beam_20_elem.msh'
m = amfe.Mesh()
m.import_msh(msh_file)

ax= amfe.plot_mesh(m)
ax.axis('equal')
ax.legend(loc=2)

Python was not able to load the fast fortran assembly routines.

Python was not able to load the fast fortran material routines.



<IPython.core.display.Javascript object>

In [2]:
# creating a mechanical component
my_comp = amfe.MechanicalSystem()
my_comp.set_mesh_obj(m)

rho = 7.86E3 # kg/mm3
E = 210.0E9 # MPa = N/mm2
my_material = amfe.KirchhoffMaterial(E=E, nu=0.3, rho=rho, thickness=1, plane_stress=True)
my_comp.set_domain(6,my_material)

map_dict = {'d': {'tag_value': 1, 'direction' : 'xy'},
            'n': {'tag_value': 12, 'direction' : 'y'},
            'c': {'tag_value': 11, 'direction' : 'y'}}
                  
s = build_SelectionOperator(my_comp,map_dict)
K, f = my_comp.assembly_class.assemble_k_and_f()
M = my_comp.assembly_class.assemble_m()

K_red = s.assemble_matrix(K,['n','c','i'])
M_red = s.assemble_matrix(M,['n','c','i'])
C_red = 0.001*K_red + 0.0*M_red
f_red = s.assemble_vector(f,['n','c','i'])

Bc_red = s.reduced_selector.build_B('c')
Bn_red = s.reduced_selector.build_B('n')
Newton_dofs = s.reduced_selector.selection_dict['n']
Amplitude = 5
f_red[Newton_dofs] = Amplitude


print('Number of nodes is equal to %i' %my_comp.mesh_class.no_of_nodes)

Preallocating the stiffness matrix
Done preallocating stiffness matrix with 20 elements and 66 dofs.
Time taken for preallocation: 0.00 seconds.
WARNING! This function will be deprecate. Please import this function from utils
WARNING! This function will be deprecate. Please import this function from utils
WARNING! This function will be deprecate. Please import this function from utils
Removing dofs related to label d. Setting dof values equal to 0
Removing dofs related to label d. Setting dof values equal to 0
Number of nodes is equal to 33


C:\Users\ge72tih\AppData\Local\Continuum\anaconda3\envs\ndcsi\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [3]:
#HBM variables
nH = 3
n_points = 1000
n_dofs = len(f_red)
beta = 1.0
B_delta = np.array([[-1, 1],
                    [-1, 1]])

H = np.array([[-1, 0],
              [ 0, 1]])

Q = assemble_HBMOperator(n_dofs,number_of_harm=nH ,n_points=n_points)
Tc = Bc_red.toarray()

In [4]:
f_aug = list(f_red)
f_aug.extend(list(f_red*0.0)*(nH-1))
f_aug = np.array(f_aug)
fl = Q.dot(f_aug).real
fl_ = Q.H.dot(fl) # force in frequency domain
fnl = lambda u : Bc_red.T.dot(Bc_red.dot(beta*((u)**3)))
fnl_ = lambda u_ : Q.H.dot(fnl(Q.dot(u_))) - fl_
Z = lambda w : create_Z_matrix(K_red,C_red,M_red,f0= w/(2.0*np.pi),nH=nH, static=False)
R = lambda u_, w : Z(w).dot(u_) + fnl_(u_)

In [5]:
%%time
x0 = np.array([0.0]*n_dofs*nH,dtype=np.complex)
y_d, p_d, info_dict = optimize.continuation(R,x0=x0,p_range=(0.01,800.0), p0=1, correction_method='moore_penrose',
                                            max_int=10000, max_dp=5,step=1.0, max_int_corr=30, tol=1.0E-10)



c:\amfe\amfe\frequency_module\frequency.py:410: RuntimeWarning: divide by zero encountered in true_divide
  T = two_pi/w0


KeyboardInterrupt: 

In [6]:
len(p_d)

NameError: name 'p_d' is not defined

In [ ]:
plt.figure()
plt.plot(p_d.real,np.abs(y_d[0,:]),'k.')

In [ ]:
import dill as pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename, 'rb') as input:
        obj = pickle.load(input)
    return obj

In [ ]:
save_object(y_d,'y_d.pkl')

save_object(p_d,'p_d.pkl')

In [ ]:
yy = load_obj('y_d.pkl')

In [ ]:
plt.figure()
plt.plot(np.abs(yy.T[:,0]),'.')

In [ ]:
%%time
freq_list = np.linspace(1,200.0,100)
u_list = []
for w in freq_list:
    opt_obj = optimize.root(lambda u_ : R(u_,w),x0=x0)
    u_list.append(opt_obj.x)
    


In [ ]:
dod_id=0
plt.figure()
plt.plot(freq_list,np.abs(np.array(u_list).T[dod_id,:]),'o')

In [ ]:
eigval, eigvec = sparse.linalg.eigs(K_red,6,M_red,which='SM')

In [ ]:
np.sort(np.sqrt(eigval))

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
from ipywidgets import interact
fig2, (ax21,ax22) = plt.subplots(1,2,figsize=(10,5))
#fig2, ax21 = plt.subplots(1,1,figsize=(10,5))
ax21.set_ylim([-4,6])
ax21.set_xlim([0,10])
ax21.axis('equal')
time_inc_list = list(range(0,n_points))

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                    wspace=0.35)

contact_dof = s.selection_dict['c'][0]
contact_dof_in_time = np.array(my_comp.u_output).T[contact_dof]
max_contact_disp = abs(contact_dof_in_time).max()

def update(u_id=0,factor=100):
    global ax21
    global ax22
    
    ax21.clear()
    ax22.clear()
    

    u_contact_disp = contact_dof_in_time[u_id]
    x = contact_dof_in_time[:u_id+1]
    t = time_inc_list[:u_id+1]
    
    ax22.plot(t,x,'r-')
    ax22.plot(u_id,u_contact_disp,'yo')
    l, ax21 = amfe.plot_2D_system_solution(my_comp,u_id=u_id,factor=factor,ax=ax21,highlight_nodes=[1])
    #ax2.axis('equal')
    ax21.set_xlabel('x [mm]')
    ax21.set_ylabel('y [mm]')
    ax22.set_title('Max displacement = %1.3e' %max_contact_disp)
    ax22.set_ylabel('Y displacement [mm]')
    ax22.set_xlabel('time increment')
    ax22.set_ylabel('Y displacement [mm]')
    p = 1.5
    ax22.set_ylim([-p*max_contact_disp,p*max_contact_disp])
    ax22.set_xlim([0,n_points])
    ax21.set_ylim([-2,2])
    ax21.set_xlim([-1,11])
    
    
    return l
    
ani = FuncAnimation(fig2, update, frames=np.arange(0, n_points ,2), blit=True, interval=1)    
#update(u_id=0)


In [ ]:
def cubic_string(beta):
    
    return lambda u : beta*(u**3)

def build_nonlinear_force_in_freq(fnl,Q,Bc):
    
    return lambda u_ : Bc.T.dot(Bc.dot(Q.T.dot((fnl(Q.dot(u_))))))
    
def build_nonlinear_force_in_freq_with_Q_red(fnl,Q_red):
    
    return lambda u_ : Q_red.T.dot((fnl(Q_red.dot(u_))))
    
def build_residual(Z,fnl_,fl_):
    ''' 
    Parameters
        Z : np.array
            Dynamic stiffness matrix
        fnl : lambda function
            
        
    '''
    
    return lambda u_ : Z.dot(u_) + fnl_(u_) - fl_

# Harmonic Analysis
K_red = s.assemble_matrix(K,['n','c','i'])
M_red = s.assemble_matrix(M,['n','c','i'])
C_red = 0.001*K_red + 0.0*M_red
f = f_one_dof_.real*s.build_B('n').toarray().flatten()
f_red = s.assemble_vector(f,['n','c','i'])

Bc_red = s.reduced_selector.build_B('c')
Bn_red = s.reduced_selector.build_B('n')

f0 = 30.0
number_of_harm=10
value = -100000
n_points = 300
beta = 1000.0
complex_data= False
static=False

force_in_time = frequency.linear_harmonic_force(a=value, f0 = f0, n_points=n_points, cos=True)
q = frequency.hbm_complex_bases(f0,number_of_harm=number_of_harm,n_points=n_points, static=static, complex_data= complex_data)
Z_red = frequency.create_Z_matrix(K_red,C_red,M_red,f0=f0,nH=number_of_harm, static=static, complex_data= complex_data)

fl_1dof_ = q.T.dot(force_in_time) # creating 1 dof force in Frequency domain

u_dym_red = sparse.linalg.spsolve(K_red,f_red)


# building multidimentional Harmonic Basis
amplitute_dim = K_red.shape[0] 
I = sparse.eye(amplitute_dim)

Q = sparse.kron(I,q)
I_harm = sparse.eye(number_of_harm)
Bc_aug = sparse.kron(I_harm, Bc_red)
Bn_aug = sparse.kron(I_harm, Bn_red)
Q_red = sparse.kron(Bc_red ,q)
#fl_aug_ = sparse.kron(I_harm, fl_)


fnl = cubic_string(beta)
fnl_ = build_nonlinear_force_in_freq(fnl,Q,Bc_aug)
fnl2_ = build_nonlinear_force_in_freq_with_Q_red(fnl,Q_red)
fl_ = Bn_aug.T.dot(fl_1dof_) # creating multiple dofs force in Frequency domain
R_func = build_residual(Z_red,fnl_,fl_)

#ul_ = sparse.linalg.spsolve(Z_red,fl_)
u_ = np.random.rand(Z_red.shape[0])
R_func(u_)
opt_obj = optimize.root(R_func,u_,method='krylov')
x0 = opt_obj.x

In [ ]:
#I_red = sparse.hstack([I]*number_of_harm) # create matrix for sumation of amplitudes
u_dir = np.array(len(s.selection_dict['d'])*[0])
get_amplitute_by_t_id = lambda t_id : sparse.kron(q[t_id,:],I).dot(x0)

u_ndym = []
for t_id in range(n_points):
    unl_ = get_amplitute_by_t_id(t_id)
    u = np.append(u_dir,unl_)    
    u_ndym.append(s.P.T.dot(u)) # back to original order

#time_list = list(np.arange(0,1/(f0), 0.0005))
#u_time_list = list(map(lambda t : np.cos(2.0*np.pi*f0*t)*u_ndym, np.arange(0,1/(f0), 0.0005)))


In [ ]:
my_comp.u_output = []
my_comp.u_output.extend(u_ndym)

from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
fig3, (ax31,ax32) = plt.subplots(1,2,figsize=(10,5))
#fig2, ax21 = plt.subplots(1,1,figsize=(10,5))
ax31.set_ylim([-4,6])
ax31.set_xlim([0,10])
ax31.axis('equal')

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                    wspace=0.35)

contact_dof = s.selection_dict['c'][0]
contact_dof_in_time = np.array(my_comp.u_output).T[contact_dof]
t = list(range(0,len(contact_dof_in_time)))
u_max = max(contact_dof_in_time)
def update2(u_id=0,mult=0.1):
    global ax21
    global value
    ax31.clear()
    ax32.clear()
    
    u_contact_disp = contact_dof_in_time[u_id]
    #mult = 1000/abs(value) 
    x =  1.0*contact_dof_in_time[:u_id+1]
    time_slice = t[:u_id+1]
    ax32.plot(time_slice,x,'r-')
    #ax32.plot(contact_dof_in_time,'r-')
    ax32.plot(time_slice[-1],x[-1],'yo')
    l, ax31x = amfe.plot_2D_system_solution(my_comp,u_id=u_id,factor=mult, ax=ax31, highlight_nodes=[1])
    #ax2.axis('equal')
    ax32.set_xlabel('time (unit)')
    ax32.set_ylabel('Y displacement (mm)')
    ax32.set_ylim([-u_max*1.5,u_max*1.5])
    ax32.set_xlim([0,len(contact_dof_in_time)])
    
    return l
    
ani = FuncAnimation(fig3, update2, frames=np.arange(0, len(contact_dof_in_time) ,2), blit=True, interval=1)    
#update2(0)

In [ ]:
N = n_points
T = 1/(f0)
dt = T/(N-1)
fft_obj = frequency.Fourier(T,dt)
fft_obj.freq_list

plt.figure()
plt.plot(fft_obj.freq_list,np.abs(fft_obj.fourier_transform(contact_dof_in_time)),'b--')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude [mm]')

In [ ]:
plt.figure()
plt.plot(fft_obj.freq_list,np.angle(fft_obj.fourier_transform(contact_dof_in_time)),'b--')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Shift [mm]')